# Master Thesis Proposal 
# Biot-Savart Law for the Magnetization 
# in Thin-Walled Steel Structures

## Section 1: Introduction 

- goal of paper: solve the Biot-Savart for the magnetization vector ${\mathbf M}$; 
- motivation: consider thin-walled steel structures immersed in earth magnetic field. The wish is to render these structures non-detectable for radar (and thus incoming electromagnetic fields). This is accomplished by applying [degaussing](https://en.wikipedia.org/wiki/Degaussing); 
- computing magnetization ${\mathbf M}$ induced in the steel structure by external field (earth magntic field); 
- eembedding in broader context of integral equations in magnetic fields; 
- traditional approach in method of moments: derive under integral sign. Choice of the basis function: Morandi-2009 chooses Dirac delta functions (collocation).
- our approach:  We wish to generate the Method of Moment (MoM) matrix as the linear transformation matrix (sensitivity matrix, input-output matrix) mapping magnetization vector $\vec{M}(\vec{x})$ to the magnetic flux vector $\vec{B}(\vec{x})$ after spatial discretization. Thus first integrate, then differentiate. These ideas are borrowed from non-linear FEM analysis is which the Jacobian is formed as the derivative of the residual vector w.r.t. the state vector.
 - advantages: (1/2) avoid increasing singularity of integral and (2/2) allow for more intricate basis functions; 
- implementation in Julia b composing functionality for adaptive integration (for step-1) and automatic differentiation (for step-2); 
- paper limitation: model is assumed to be **linear**. The effect of saturation and hysteresis will be neglected. No current sources are included; 
- implementation in Julia: cite literature to relevant components implemented in Julia; 
- This paper is structured as follows: ... 

## Section 2: Inverting the Biot-Savart Law for the Magnetization 

<b>Problem Formulation</b> Assume a 3D computational domain $\vec{x} \in \Omega_m$ on which ${\mathbf M} \neq {\mathbf 0}$. $\Omega_m$ typically represents a metallic plate in an external magnetic field. We assume that no external current sources are present and that the magnetic material reacts accordinf to a linear magnetic material. In a continuous setting (i.e. prior to spatial discretization), the magnetic vector potential $\vec{A}(\vec{x})$ is a function of the magnetization vector $\vec{M}(\vec{x})$. A [Biot-Savart](https://en.wikipedia.org/wiki/Biot–Savart_law) type relation yields that 
$$\vec{A}(\vec{x}) = \frac{\mu_0}{4 \pi} \int_{\Omega_m} 
                \frac{\vec{M}(\vec{x}') \times (\vec{x}-\vec{x}')}{\|\vec{x}-\vec{x}'\|^3} \, d\vec{x}' \, = \int_{\Omega_m} \vec{M}(\vec{x}') \times K(\vec{x}, \vec{x}') \, d\vec{x}' \, , $$ where the kernel $K(\vec{x}, \vec{x}')$ is given by 
$$K(\vec{x},\vec{x}') = \frac{\mu_0}{4 \pi}  
                \frac{(\vec{x}-\vec{x}')}{\|\vec{x}-\vec{x}'\|^3} = 
                \frac{\mu_0}{4 \pi} \nabla \frac{1}{\|\vec{x}-\vec{x}'\|} \, . $$ 
Properties of $K(\vec{x},\vec{x}')$ are extensively studied in the literature. 
The magnetic flux $\vec{B}(\vec{x})$ (should this read $\vec{B}_M(\vec{x})$ instead) is then obtained via 
$$ \vec{B}(\vec{x}) = \nabla \times \vec{A}(\vec{x}) \, . $$
The magnetic field $\vec{H}(\vec{x})$ can be obtain as $\mu \, \vec{B}(\vec{x})$ in the ferromagnetic region and from $\mu \, \vec{B}(\vec{x}) + \vec{M}(\vec{x})$ in the magnetic region.  
Thus something like (not entirely sure)
$$ {\mathbf M} = \frac{\chi}{\mu} \, [ {\mathbf B}_M + {\mathbf B}_J ] 
  = \frac{\mu_0 \, \chi}{4 \, \pi \, \mu} \, \nabla \times \int_{\tau_M} 
    \frac{ \mathbf{M}(\mathbf{r}') \times \left[ \mathbf{r} - \mathbf{r}' \right]}
         {\| \mathbf{r} - \mathbf{r}'\|^3} d\mathbf{r}' $$
         
<b>Spatial Discretization</b> A mesh $\Omega^h$ on $\Omega$ with $N$ nodes is defined. The numerical approximation to vector $\vec{M}(\vec{x})$ is expressed as a linear combination on shape function (or basis functions) as (three components of $\vec{M}^h(\vec{x}$ expressed in the same linear nodal first order Lagrange basis functions, the summation upper therefore denoted by $3N$)
$$ 
\vec{M}(\vec{x}) \thickapprox \vec{M}^h(\vec{x}) 
= \left( \sum_{i=1}^{N} m^x_i \, \phi_i(\vec{x}), \sum_{i=1}^{N} m^y_i \, \phi_i(\vec{x}),
          \sum_{i=1}^{N} m^z_i \, \phi_i(\vec{x}) \right) 
\, .$$ 
Here we expand the three components of the vector $\vec{M}(\vec{x})$ in the same function basis. (notation changes in case that piecewise constant per element basis functions are used). 
Replacing $\vec{M}(\vec{x})$ by $\vec{M}^h(\vec{x})$ in the expression for $\vec{A}(\vec{x})$ yiels the numerical approximation $\vec{A}^h(\vec{x})$. Similarly, replacing $\vec{A}(\vec{x})$ by $\vec{A}^h(\vec{x})$ in the expression for $\vec{B}(\vec{x})$, yiels the numerical approximation $\vec{B}^h(\vec{x})$. Let $\vec{m}$ denote the $3N$-vector of expansion coefficients of $\vec{M}^h(\vec{x})$ in the basis of $\phi_i{(\vec{x})}$ for $1 \leq i \leq N$ shape functions. Let $\vec{a}$ and $\vec{b}$ denote the $3N$-vectors obtained by point-matching the relationships that define them. Then clearly both $\vec{a}$ and $\vec{b}$ are vector-valued functions with $3N$ components that map $\vec{m}$ to $\vec{a}(\vec{m})$ and $\vec{b}(\vec{m})$, respectively. We can thus write formally that $\vec{a}: \vec{m} \in R^{3N} \rightarrow \vec{a}(\vec{m}) \in R^{3N}$ and  
$\vec{b}: \vec{m} \in R^{3N} \rightarrow \vec{b}(\vec{m}) \in R^{3N}$. The Jacobian of mapping 
$\partial b_i / \partial m_j$ is the MoM matrix. 

<b>Generation of the MoM matrix</b> We wish to develop a procedure that compute this Jacobian by going through the following three steps: 
1. apply (adaptive) quadrature (including handling of singular integrals) to compute the mapping $\vec{a}(\vec{m})$, i.e., a vector function of $3N$ components $a_i$ that each depend on the $3N$ vector $\vec{a}$. The quadrature on $\Omega$ is carried out element-wise;
2. compute $\vec{b} = \nabla \times \vec{a}(\vec{x})$  by applying automatic differentiation to differentiate wrt to $x$, $y$ and $z$
3. apply automatic differentiation to differentiate (compute the Jacobian) the magnetic flux $\vec{b}(\vec{m}) = \vec{b}(\vec{a}(\vec{m})))$ wrt to $\vec{m}$ to obtain the MoM matrix;

Solving of the linear system with the MoM matrix is considered outside of scope of this paper. 

## Section 3: Continuous Problem Formulation 

### No External Current Sources 

See e.g. [magnetism](https://en.wikipedia.org/wiki/Magnetism). 

$ \nabla \times {\mathbf H} = {\mathbf 0} \hspace{.5cm} \nabla \cdot {\mathbf B} = 0 \hspace{.5cm} 
  {\mathbf B} = \mu_0 \, {\mathbf H + \mathbf M}$
  
Magnetic scalar potential $\phi_m({\mathbf x})$ such that ${\mathbf H} = - \nabla \phi_m$. 

Poisson equation for $\phi_m$ given by $ - \nabla \cdot [ \mu \, \nabla \phi_m ] = \nabla \cdot {\mathbf M}$ supplied with boundary conditions.  

Integral equation: $ \int_{\Omega} G(x,s) \, u(s) \, ds = f(x)$ where $f(x) = $ where $G(x,s) = \nabla \cdot {\mathbf M}$ is the [Green function](https://en.wikipedia.org/wiki/Green's_function). 

Upon introducing the magnetic susceptibility

$ {\mathbf M} = \chi \, {\mathbf H} = \frac{1}{\mu} \, \chi \, {\mathbf B} = 
  \frac{1}{\mu} \, \chi \, [ {\mathbf B}_M + {\mathbf B}_J ]  $ and thus 

$ \mu_0 [ \mathbf{H} + \mathbf{M} ] = \mu_0(1 + \chi) \,\mathbf{H} = 
  \mu_r \, \mu_0 \, \mathbf{H} = \mu \, \mathbf{H} $ 

### With External Current Sources 

## Section 4: Discrete Problem Formulation 

### No External Current Sources

### With External Current Sources

Weighted residual approach. Expansion of solution in set of basis vectors. 

Face basis function: $W_f({\mathbf r}) = \int_{S_f} \frac{1}{\| \mathbf{r} - \mathbf{r}' \|} d\mathbf{r}' $

Gradient of face basis function: $\nabla W_f({\mathbf r}) = \sum_{\ell_e \in S_f} {\mathbf n}_f \times {\mathbf t}_e \, w_e(\mathbf{r}) 
                           - {\mathbf n}_f \, \Omega_f(\mathbf{r})$ 
                           
Edge basis function for $e$ connecting ${\mathbf r}_1$ and ${\mathbf r}_2$: 
$w_e({\mathbf r}) = \log \frac{\|\mathbf{r}_2 - \mathbf{r}\| + \|\mathbf{r}_1 - \mathbf{r}\| +
                               \|\mathbf{r}_2 - \mathbf{r}_1\|}
                              {\|\mathbf{r}_2 - \mathbf{r}\| + \|\mathbf{r}_1 - \mathbf{r}\| -
                               \|\mathbf{r}_2 - \mathbf{r}_1\|}$  
                               
Solid angle subtended to face $S_f$ as seen from point ${\mathbf r}$ $\Omega_f$.                                

## Section 5: Implementation  

### Geometry Definition 

Create STL-file (or equivalent) of the geomtry using Blender (or equivalent). 

### Mesh Generation 

Use Gmsh (or equivalent) to generate the mesh. Either import the STL-file into the GUI of GMSH or use the plugin for Gmsh in Blender; 

### Choice of the Basis Functions 

Nodal vs. edge basis functions of a given polynomial order. 
Collocation vs. Galerkin. (Non)-symmetry of the coefficients matrix. 

### Quadrature and cubature 

Various ways to implementation [quadrature](https://en.wikipedia.org/wiki/Numerical_integration) for one-dimensional and higher-dimensional integrals (cubature) 
1. [Integrals.jl](https://juliahub.com/ui/Packages/General/Integrals);
2. [QuadQK.jl](https://juliamath.github.io/QuadGK.jl/stable/);
3. [HCubature.jl](https://github.com/JuliaMath/HCubature.jl); 
4. [IteratedIntegration.jl](https://github.com/lxvm/IteratedIntegration.jl/tree/main); 
5. [Discourse on integration of functions over geometries](https://discourse.julialang.org/t/integration-of-functions-over-geometries/81994/21); 

### Generate the Coefficient Matrix and the Right-Hand Side Vector 

Import the mesh into Julia and perform loop over the elements to construct the right-hand side vector. 
[Automatic differentiation](automatic_differentiation.ipynb) and [additional_automatic_diffferentiation](additional_automatic_diffferentiation.ipynb) for assembly. 

### Post-Processing 

Write solution to STL-file and use Paraview to view the computed solution. Use of [writeVTK](https://github.com/JuliaVTK/WriteVTK.jl) or higher in abstraction level such as [gridap.Visualization](https://gridap.github.io/Gridap.jl/stable/Visualization/). 

## Section 6: Test Cases 

### Section 1.6: Classical FEM Stiffness Matrix Assembly using AD 

See notebook [fem_matrix_assembly_using_ad](fem_matrix_assembly_using_ad.ipynb).

### Section 2.6: Spherical Shell  
No end effects. Analytical reference solution. Define spherical coordinate system. Define external magnetic field.  $\vec{A} = (0, A_{\theta}(r,\phi),0)$ where 
$$A_{\theta}(r,\phi) = (-\beta \frac{r}{2} + \gamma \frac{1}{r^2}) \, \sin(\phi)$$ and 
$\vec{M} = (M_r(r,\phi),0,M_{\phi}(r,\phi))$ where 

$$ M_r(r,\phi) = \frac{1}{r \, \sin(\phi)} \left[ \frac{\partial}{\partial \phi} (A_{\theta} \sin(\phi) )   \right] = (-\beta + \gamma \frac{2}{r^3}) \, \cos(\phi) $$

$$ M_{\phi}(r,\phi) = \frac{1}{r} \left[ - \frac{\partial}{\partial r} (A_{\theta} r ) \right] 
   = (\beta + \gamma \frac{1}{r^3}) \, \sin(\phi) $$
   
See notebook [spherical-shell-test-case](spherical-shell-test-case.ipynb).   

### Section 3.6:  2D Rectangular Plate
With end effects.
See notebook [2d_magnetized_plate_test_case](2d_magnetized_plate_test_case.ipynb). 


### Section 4.6:  3D Rectangular Plate 
See notebook [3d_magnetized_plate_test_case](3d_magnetized_plate_test_case.ipynb). 

###  Section 5.6: Ship M4 Mock-up 

## Section 7: Publically Available Software Components

1. [BEAST](https://github.com/krcools/BEAST.jl) and example of Magnetic Field Integral Equation (MFIE) [mfie.jl](https://github.com/krcools/BEAST.jl/blob/master/examples/mfie.jl) 
2. [HMatrices](https://github.com/WaveProp/HMatrices.jl) and [related documentation](https://waveprop.github.io/HMatrices.jl/dev/)
3. [PETSc](https://github.com/JuliaParallel/PETSc.jl) 
4. [iterativesolvers](https://iterativesolvers.julialinearalgebra.org/stable/), [KrylovKit](https://github.com/Jutho/KrylovKit.jl) or any of the alternatives mentioned by the latter
5. [LowRankApprox](https://github.com/JuliaLinearAlgebra/LowRankApprox.jl): provides low-rank approximations; 
6. [ACAFact](https://github.com/cgeoga/ACAFact.jl): adaptive cross-approximation. References Bebendorf-Rjasanov-2002. This paper uses the TEAM-10 as benchmark. 

## References 

1. Morandi-2010 
2. Fabbri-2009 
3. book Mario Bebendorf Springer-2003